In [ ]:
import automatization
from automatization.ple_statistics import PleAuto
import pandas as pd
import os
import time
import json
from importlib import reload
import numpy as np
from automatization.ple_statistics import PleAuto
from matplotlib import pyplot as plt
reload(automatization.ple_statistics)
from automatization.ple_statistics import PleAuto
from scipy.constants import speed_of_light
import pickle
from pathlib import Path

pa = PleAuto(
    ple_gui, 
    ple_optimize_logic, 
    laser_scanner_logic,
    poi_manager_logic, 
    scanning_optimize_logic, 
    spectrometer, 
    spectrometerlogic,
    cobolt,
    ibeam_smart
)


In [ ]:

def ple_is_here(res, center_sigma = 3e3, amplitude = 1000, sigma_stderr_ratio = 4, amplitude_stderr_ratio=3):
    it_is = True
    if res["center"].stderr is None or res["sigma"].stderr is None or res["sigma"].value is None:
        return False
    if ( res["center"].stderr > center_sigma or 
    res["sigma"].stderr * sigma_stderr_ratio > res["sigma"].value or 
    res["amplitude"].stderr * amplitude_stderr_ratio > res["amplitude"].value or
    res["amplitude"].value < amplitude): 
        #ple is gone.
        return False

    return it_is

# constraints
scan_range_constr = laser_scanner_logic.scanner_constraints.axes["a"].value_range
scanning_optimize_logic._backwards_line_resolution = 20

# high_finesse_wavemeter_remote.start_acquisition()
# switchlogic.set_state("ScanningMode", 'NI')
# time.sleep(1)
# # factor_sigma = 0.509
# pa.go_to_ple_target(scan_range_constr[0])
# time.sleep(5)
# w1 = high_finesse_wavemeter_remote.get_current_wavelength()
# pa.go_to_ple_target(scan_range_constr[1])
# time.sleep(5)
# w2 = high_finesse_wavemeter_remote.get_current_wavelength()
# factor_sigma = (w2 - w1) * 1000 / 26 #GHz 
factor_sigma = 1.05

Experiment

In [38]:
ple_gui

NameError: name 'ple_gui' is not defined

In [46]:
cobolt.disable_modulated()

In [10]:
cobolt.enable_modulated()
cobolt.set_laser_modulated_power(power = 2)

In [ ]:
import sys

sys.setrecursionlimit(10000)

In [1]:
ibeam_smart.enable()


In [2]:
ibeam_smart.disable()

In [6]:
cobolt.disable_modulated()

In [5]:

cobolt.enable_modulated()
cobolt.set_laser_modulated_power(power = 10)

In [ ]:
#prepare
#switch off green and turn on the 405 nm repump
ibeam_smart.disable()

cobolt.enable_modulated()
cobolt.set_laser_modulated_power(power = 5)

center_v = -7.5#laser_controller_remote.etalon_voltage
red_v = center_v + 0.5
blue_v = center_v - 0.5
eta_volts = [center_v, red_v, blue_v]
laser_controller_remote.etalon_voltage = center_v


results = []

power_steps = 3 * np.logspace(1.5, 2, 10, endpoint=True).astype(int)[::-1]
folder = r"C:\Users\yy3\Documents\data\Vlad\26-03-2023\158\#1_D\defects"
folder = os.path.join(folder, r"auto_3")

#poi_manager_logic.poi_names
poi_name = "def1"
#for poi_name in poi_manager_logic.poi_names:
if poi_name == "ref":
    raise
if not os.path.exists(folder):
    os.mkdir(folder) 


results_poi = {}
os.mkdir(folder_defect := os.path.join(folder, poi_name))
results_poi["poi_name"] = poi_name
results_poi["factor_sigma"] = factor_sigma

    
# go to a defect
pa.set_resonant_power(power = 300)
time.sleep(1)

#configure slow scanning for the wavemeter scanning optimizations
switchlogic.set_state("ScanningMode", 'Wavemeter')
# x - y - z- fine scan to find the resonance
seqs = {str(seq): idx for idx, seq in enumerate(scanner_gui._osd.settings_widget.available_opt_sequences)}
scanner_gui._osd.settings_widget.optimize_sequence_combobox.setCurrentIndex(seqs["x, y, z"])

scanner_gui._osd.change_settings({'scan_frequency': {"x": 5, "y": 5, "z": 5},
                                "scan_resolution": {"x": 80, "y": 80, "z":80},
                                "scan_range": {"x": 2.5e-6, "y": 2.5e-6, "z": 4.5e-6}})
scanner_gui._osd.accept()

time.sleep(1)
pa.go_to_poi(poi_name, opt_times=1, ref_poi=None)


switchlogic.set_state("ScanningMode", 'NI')
time.sleep(0.2)

#Check how the PLE look like
res = pa.do_ple_scan(lines = 1)
pa.save_ple(tag = "full_range",
        poi_name=poi_name, folder_name=folder_defect)
time.sleep(0.5)

for kk in range(2):
    if not ple_is_here(res, amplitude = 1500):
        time.sleep(1)
        pa.go_to_poi(poi_name, opt_times=1, ref_poi=None) 
        #Check how the PLE look like
        res = pa.do_ple_scan(lines = 1)
        pa.save_ple(tag = "full_range",
                poi_name=poi_name, folder_name=folder_defect)
        time.sleep(0.5)
    else:
        break

sigma_errs = []
for eta_v in eta_volts:
    laser_controller_remote.etalon_voltage = eta_v
    time.sleep(0.5)
    res = pa.do_ple_scan(lines = 1)
    time.sleep(0.5)
    sigma_errs.append(res["sigma"].stderr if res["sigma"].stderr is not None else 1e15)
laser_controller_remote.etalon_voltage = np.min(sigma_errs)
res = pa.do_ple_scan(lines = 1)
pa.save_ple(tag = "full_range_eta_adjusted",
                poi_name=poi_name, folder_name=folder_defect)

results_poi.update({
    "ple_scans" : {
    "full_range_first" : ple_data_logic.last_saved_files_paths 
        }
    })


#since there is a PLE -- optimize_all
pa.go_to_ple_target(res["center"].value)
#get the center wavelength:
pa.optimize_ple()
time.sleep(0.5)
wavelength = high_finesse_wavemeter_remote.get_current_wavelength()
results_poi["wavelength, THz"] = float(wavelength)

time.sleep(0.5)


scanner_gui._osd.settings_widget.optimize_sequence_combobox.setCurrentIndex(seqs["xy, z"])
scanner_gui._osd.change_settings({'scan_frequency': {"x": 25, "y": 25, "z": 25},
                                    "scan_resolution": {"x": 15, "y": 15, "z":80},
                                    "scan_range": {"x": 1e-6, "y": 1e-6, "z": 4e-6}})

scanner_gui._osd.accept()
time.sleep(0.5)
pa.optimize_all()

#Now we now where the defect is
results_poi["xyz"] = tuple(float(coord) for coord in poi_manager_logic.poi_anchors[poi_name])

res = pa.do_ple_scan(lines = 1)
time.sleep(1)
pa.save_ple(tag = f"optimized_PLE_scan_full_range",
            poi_name=poi_name, folder_name=folder_defect)

results_poi.update({
    "ple_scans" : {
    "full_range" : ple_data_logic.last_saved_files_paths 
        }
    })


In [ ]:
import sys

sys.setrecursionlimit(10000)

In [ ]:

pa.set_resonant_power(power = power_steps[1])

time.sleep(1)
res = pa.do_ple_scan(lines = 1)

fine_scan_range = ( 
    res["center"].value - 8 * res["sigma"].value ,
    res["center"].value + 8 * res["sigma"].value
                )
time.sleep(1)
res = pa.do_ple_scan(lines = 1, in_range=fine_scan_range)

scan_range_sat = ( 
    res["center"].value - 6 * res["sigma"].value ,
    res["center"].value + 6 * res["sigma"].value
                )
time.sleep(1)
res = pa.do_ple_scan(lines = 1, in_range=scan_range_sat)
scan_range_sat = ( 
    res["center"].value - 6 * res["sigma"].value ,
    res["center"].value + 6 * res["sigma"].value
                )
scan_range_sat_sigma = res["sigma"].value
time.sleep(1)

pa.save_ple(tag = "4sigma_range",
            poi_name=poi_name, folder_name=folder_defect)
time.sleep(0.5)

results_poi.update({
    "ple_scans" : {
    "4sigma_range" : ple_data_logic.last_saved_files_paths 
    }
})


# now we run saturation with a cw repump
os.mkdir(saturation_folder := os.path.join(folder_defect, "saturation"))
results_poi["saturation"] = {}
idx_no_ple = None

# #405 repump
cobolt.enable_modulated()
cobolt.set_laser_modulated_power(power = 5)
#cobolt.disable_modulated() #leave only 450nm ON
for idx, power in enumerate(power_steps):
    os.mkdir(power_folder := os.path.join(saturation_folder, f"{power}"))
    pa.set_resonant_power(power = power)
    time.sleep(1)
    #fine_scan_range = scan_range_sat
    res = pa.do_ple_scan(lines = 1, in_range=scan_range_sat)
    fine_scan_range = ( 
        res["center"].value - 4 * scan_range_sat_sigma ,
        res["center"].value + 4 * scan_range_sat_sigma
                    )
    time.sleep(0.5)
    res = pa.do_ple_scan(lines = 1, in_range=fine_scan_range)
    fine_scan_range = ( 
        res["center"].value - 4 * scan_range_sat_sigma ,
        res["center"].value + 4 * scan_range_sat_sigma
                    )
    time.sleep(0.5)
    res = pa.do_ple_scan(lines = 2, in_range=fine_scan_range)
    time.sleep(1)
    #save_results
    pa.save_ple(tag = f"{power}",
        poi_name=poi_name, folder_name=power_folder)
    #save_plots
    if res["center"].stderr is None or res["sigma"].stderr is None or res["sigma"].value is None:
        idx_no_ple = idx
        break
    if res["center"].stderr > 1e3 or res["sigma"].stderr * 4 > res["sigma"].value or res["amplitude"].stderr * 3 > res["amplitude"].value or res["amplitude"].value < 3000: 
        #ple is gone.
        idx_no_ple = idx
        break

    
    #get the center wavelength:
    
    pa.go_to_ple_target(res["center"].value)
    pa.optimize_ple()
    wavelength = high_finesse_wavemeter_remote.get_current_wavelength()

    # #charge_lifetime:
    # for i in range(10):
    #     pa.optimize_ple()
    #     time.sleep(5)

    results_poi.update({"saturation": 
                        {f"{power}":
                            {"scan_data": ple_data_logic.last_saved_files_paths,
                            "sigma": res["sigma"].value,
                            "sigma_stderr": res["sigma"].stderr,
                            "wavelength": wavelength
                            }
                            }})
if idx_no_ple is None:
    raise
    
# # photostability study
os.mkdir(photostability_folder := os.path.join(folder_defect, "photostability"))
results_poi["photostability"] = {}
#for color in ["violet", "blue", "green"]:


color = "violet"
idx_no_ple_checked = idx_no_ple - 2 if idx_no_ple > 2 else 1
cobolt.enable_modulated()

pa.set_resonant_power(power = int(power_steps[idx_no_ple_checked])) #take the last but one visible ple power
time.sleep(1)
res = pa.do_ple_scan(lines = 1, in_range=scan_range_sat)
fine_scan_range_0 = ( 
        res["center"].value - 3 * scan_range_sat_sigma ,
        res["center"].value + 3 * scan_range_sat_sigma
                    )
time.sleep(1)
cobolt.disable_modulated()


idx_no_ple_i = idx_no_ple + 3 if len(power_steps) - 3 > idx_no_ple else len(power_steps) - 1
skiped_counter = 0
for i in range(idx_no_ple_i):
    idx_no_ple_checked = idx_no_ple_i - i
    power = int(power_steps[idx_no_ple_checked])
    pa.set_resonant_power(power = power) #take the last but one visible ple power
    #reionize with blue
    
    pa.one_pulse_repump(color)
    res = pa.do_ple_scan(lines = 1, in_range=fine_scan_range_0)
    time.sleep(0.5)
    #if not ple_is_here():
        
    fine_scan_range = ( 
        res["center"].value - 6 * res["sigma"].value ,
        res["center"].value + 6 * res["sigma"].value
                    ) 
    
    pa.one_pulse_repump(color)
    res = pa.do_ple_scan(lines = 1, in_range=fine_scan_range)
    time.sleep(0.5)
    pa.save_ple(tag = f"{power}_inited_{color}",
    poi_name=poi_name, folder_name=photostability_folder)
    results_poi.update(
        {"photostability":
                { int(power_steps[idx_no_ple_checked]) :{
                            "charge_initializer": color,
                            "scan_data": ple_data_logic.last_saved_files_paths,
                            "sigma": res["sigma"].value,
                            "sigma_stderr": res["sigma"].stderr
                }}}
                )
    if not ple_is_here(res):
        skiped_counter += 1
        continue
    fine_scan_range = ( 
        res["center"].value - 8 * res["sigma"].value ,
        res["center"].value + 8 * res["sigma"].value
                    )

    pa.one_pulse_repump(color)
    res_ = pa.do_ple_scan(lines = 10, in_range=fine_scan_range)
    #fine_scan_range = ( 
    #res["center"].value - scan_range_sat,
    #res["center"].value + scan_range_sat
    #        )
    pa.save_ple(tag = f"{power}_inited_{color}",
    poi_name=poi_name, folder_name=photostability_folder)
    time.sleep(1)
    results_poi.update({"photostability": {
                    "charge_initializer": color,
                    "power": power,
                    "scan_data": ple_data_logic.last_saved_files_paths,
                    "sigma": res["sigma"].value,
                    "sigma_stderr": res["sigma"].stderr
                    }}
                    )
    if idx_no_ple_checked  < idx_no_ple - 2 or skiped_counter == 3:
        break
    #if not ple_is_here(res_):
    #    break

# take spectrum to estimate SOC
pa.set_resonant_power(power = 0)
time.sleep(2)
ibeam_smart.enable()
cobolt.enable_modulated()
cobolt.set_laser_modulated_power(power = 100)
pa.set_resonant_power(power = 0)
time.sleep(1)
pa.take_spectrum()

pa.save_spectrum(name_tag=f"{poi_name}_blueNgreen", folder_path=folder_defect)
results_poi["spectrum_data"] = spectrometerlogic.last_saved_path

spectrometer._mw.data_widget.fit_widget.sigDoFit.emit("DoubleLorentzian")
time.sleep(0.2)
# spectrometer.fit_results.params["center_1"].value
if spectrometer.fit_results is not None:
    params = spectrometer.fit_results.params
    results_poi["SOC, GHz"] = float(speed_of_light / params["center_2"].value - speed_of_light / params["center_1"].value)

ibeam_smart.disable()
#cobolt.disable_modulated()
cobolt.enable_modulated()
cobolt.set_laser_modulated_power(power = 5)

with open(os.path.join(folder_defect, f'results_{poi_name}'), 'wb') as handle:
    pickle.dump(results_poi, handle, protocol=pickle.HIGHEST_PROTOCOL)
        

In [ ]:
laser_controller_remote.etalon_voltage = -7.5

In [ ]:
#prepare
#switch off green and turn on the 405 nm repump
ibeam_smart.disable()

cobolt.enable_modulated()
cobolt.set_laser_modulated_power(power = 5)

center_v = -7.5#laser_controller_remote.etalon_voltage
red_v = center_v + 0.5
blue_v = center_v - 0.5
eta_volts = [center_v, red_v, blue_v]
laser_controller_remote.etalon_voltage = center_v


results = []

power_steps = 3 * np.logspace(1.5, 2, 10, endpoint=True).astype(int)[::-1]
folder = r"C:\Users\yy3\Documents\data\Vlad\26-03-2023\158\#1_D\auto_2_after_manual"
folder = os.path.join(folder, r"auto_0_2")


for poi_name in poi_manager_logic.poi_names:
    if poi_name == "ref":
        continue
    if not os.path.exists(folder):
        os.mkdir(folder) 


    results_poi = {}
    os.mkdir(folder_defect := os.path.join(folder, poi_name))
    results_poi["poi_name"] = poi_name
    results_poi["factor_sigma"] = factor_sigma

        
    # go to a defect
    pa.set_resonant_power(power = 300)
    time.sleep(1)

        #configure slow scanning for the wavemeter scanning optimizations
    switchlogic.set_state("ScanningMode", 'Wavemeter')
    # x - y - z- fine scan to find the resonance
    seqs = {str(seq): idx for idx, seq in enumerate(scanner_gui._osd.settings_widget.available_opt_sequences)}
    scanner_gui._osd.settings_widget.optimize_sequence_combobox.setCurrentIndex(seqs["x, y, z"])

    scanner_gui._osd.change_settings({'scan_frequency': {"x": 5, "y": 5, "z": 5},
                                    "scan_resolution": {"x": 80, "y": 80, "z":80},
                                    "scan_range": {"x": 2.5e-6, "y": 2.5e-6, "z": 4.5e-6}})
    scanner_gui._osd.accept()

    time.sleep(1)
    pa.go_to_poi(poi_name, opt_times=1)


    switchlogic.set_state("ScanningMode", 'NI')
    time.sleep(0.2)

    #Check how the PLE look like
    res = pa.do_ple_scan(lines = 1)
    pa.save_ple(tag = "full_range",
            poi_name=poi_name, folder_name=folder_defect)
    time.sleep(0.5)

    for kk in range(3):
        if not ple_is_here(res, amplitude = 2000):
            time.sleep(1)
            pa.go_to_poi(poi_name, opt_times=1) 
            #Check how the PLE look like
            res = pa.do_ple_scan(lines = 1)
            pa.save_ple(tag = "full_range",
                    poi_name=poi_name, folder_name=folder_defect)
            time.sleep(0.5)
        else:
            break
    sigma_errs = []
    for eta_v in eta_volts:
        laser_controller_remote.etalon_voltage = eta_v
        time.sleep(0.5)
        res = pa.do_ple_scan(lines = 1)
        time.sleep(0.5)
        if ple_is_here(res, amplitude=2000):
            sigma_errs.append(res["sigma"].stderr if res["sigma"].stderr is not None else 1e15)
        else:
            sigma_errs.append(1e15)
    if min(sigma_errs) == 1e15:
        continue
    laser_controller_remote.etalon_voltage = eta_volts[sigma_errs.index(min(sigma_errs))]
    res = pa.do_ple_scan(lines = 1)
    pa.save_ple(tag = "full_range_eta_adjusted",
                    poi_name=poi_name, folder_name=folder_defect)


            #continue

    results_poi.update({
        "ple_scans" : {
        "full_range_first" : ple_data_logic.last_saved_files_paths 
            }
        })


    #since there is a PLE -- optimize_all
    pa.go_to_ple_target(res["center"].value)
    #get the center wavelength:
    pa.optimize_ple()
    time.sleep(0.5)
    wavelength = high_finesse_wavemeter_remote.get_current_wavelength()
    results_poi["wavelength, THz"] = float(wavelength)

    time.sleep(0.5)


    scanner_gui._osd.settings_widget.optimize_sequence_combobox.setCurrentIndex(seqs["xy, z"])
    scanner_gui._osd.change_settings({'scan_frequency': {"x": 25, "y": 25, "z": 25},
                                        "scan_resolution": {"x": 15, "y": 15, "z":80},
                                        "scan_range": {"x": 1e-6, "y": 1e-6, "z": 4e-6}})

    scanner_gui._osd.accept()
    time.sleep(0.5)
    pa.optimize_all()

    #Now we now where the defect is
    results_poi["xyz"] = tuple(float(coord) for coord in poi_manager_logic.poi_anchors[poi_name])

    res = pa.do_ple_scan(lines = 1)
    time.sleep(1)
    pa.save_ple(tag = f"optimized_PLE_scan_full_range",
                poi_name=poi_name, folder_name=folder_defect)

    results_poi.update({
        "ple_scans" : {
        "full_range" : ple_data_logic.last_saved_files_paths 
            }
        })


    pa.set_resonant_power(power = power_steps[1])

    time.sleep(1)
    res = pa.do_ple_scan(lines = 1)

    fine_scan_range = ( 
        res["center"].value - 8 * res["sigma"].value ,
        res["center"].value + 8 * res["sigma"].value
                    )
    time.sleep(1)
    res = pa.do_ple_scan(lines = 1, in_range=fine_scan_range)

    scan_range_sat = ( 
        res["center"].value - 4 * res["sigma"].value ,
        res["center"].value + 4 * res["sigma"].value
                    )
    time.sleep(1)
    res = pa.do_ple_scan(lines = 1, in_range=scan_range_sat)
    scan_range_sat = ( 
        res["center"].value - 4 * res["sigma"].value ,
        res["center"].value + 4 * res["sigma"].value
                    )
    scan_range_sat_sigma = res["sigma"].value
    time.sleep(1)

    pa.save_ple(tag = "4sigma_range",
                poi_name=poi_name, folder_name=folder_defect)
    time.sleep(0.5)

    results_poi.update({
        "ple_scans" : {
        "4sigma_range" : ple_data_logic.last_saved_files_paths 
        }
    })


    # now we run saturation with a cw repump
    os.mkdir(saturation_folder := os.path.join(folder_defect, "saturation"))
    results_poi["saturation"] = {}
    idx_no_ple = None

    # #405 repump
    cobolt.enable_modulated()
    cobolt.set_laser_modulated_power(power = 5)
    #cobolt.disable_modulated() #leave only 450nm ON
    for idx, power in enumerate(power_steps):
        os.mkdir(power_folder := os.path.join(saturation_folder, f"{power}"))
        pa.set_resonant_power(power = power)
        time.sleep(1)
        #fine_scan_range = scan_range_sat
        res = pa.do_ple_scan(lines = 1, in_range=scan_range_sat)
        fine_scan_range = ( 
            res["center"].value - 4 * scan_range_sat_sigma ,
            res["center"].value + 4 * scan_range_sat_sigma
                        )
        time.sleep(0.5)
        res = pa.do_ple_scan(lines = 1, in_range=fine_scan_range)
        fine_scan_range = ( 
            res["center"].value - 4 * scan_range_sat_sigma ,
            res["center"].value + 4 * scan_range_sat_sigma
                        )
        time.sleep(0.5)
        res = pa.do_ple_scan(lines = 2, in_range=fine_scan_range)
        time.sleep(1)
        #save_results
        pa.save_ple(tag = f"{power}",
            poi_name=poi_name, folder_name=power_folder)
        #save_plots
        if res["center"].stderr is None or res["sigma"].stderr is None or res["sigma"].value is None:
            idx_no_ple = idx
            break
        if res["center"].stderr > 1e3 or res["sigma"].stderr * 4 > res["sigma"].value or res["amplitude"].stderr * 3 > res["amplitude"].value or res["amplitude"].value < 3000: 
            #ple is gone.
            idx_no_ple = idx
            break

        
        #get the center wavelength:
        
        pa.go_to_ple_target(res["center"].value)
        pa.optimize_ple()
        wavelength = high_finesse_wavemeter_remote.get_current_wavelength()

        # #charge_lifetime:
        # for i in range(10):
        #     pa.optimize_ple()
        #     time.sleep(5)

        results_poi.update({"saturation": 
                            {f"{power}":
                                {"scan_data": ple_data_logic.last_saved_files_paths,
                                "sigma": res["sigma"].value,
                                "sigma_stderr": res["sigma"].stderr,
                                "wavelength": wavelength
                                }
                                }})
    if idx_no_ple is None:
        continue
        
    # # photostability study
    os.mkdir(photostability_folder := os.path.join(folder_defect, "photostability"))
    results_poi["photostability"] = {}
    #for color in ["violet", "blue", "green"]:


    color = "violet"
    idx_no_ple_checked = idx_no_ple - 2 if idx_no_ple > 2 else 1
    cobolt.enable_modulated()

    pa.set_resonant_power(power = int(power_steps[idx_no_ple_checked])) #take the last but one visible ple power
    time.sleep(1)
    res = pa.do_ple_scan(lines = 1, in_range=scan_range_sat)
    fine_scan_range_0 = ( 
            res["center"].value - 4 * scan_range_sat_sigma ,
            res["center"].value + 4 * scan_range_sat_sigma
                        )
    time.sleep(1)
    cobolt.disable_modulated()


    idx_no_ple_i = idx_no_ple + 3 if len(power_steps) - 3 > idx_no_ple else len(power_steps) - 1
    skiped_counter = 0
    for i in range(idx_no_ple_i):
        idx_no_ple_checked = idx_no_ple_i - i
        power = int(power_steps[idx_no_ple_checked])
        pa.set_resonant_power(power = power) #take the last but one visible ple power
        #reionize with blue
        
        pa.one_pulse_repump(color)
        res = pa.do_ple_scan(lines = 1, in_range=fine_scan_range_0)
        time.sleep(0.5)
        #if not ple_is_here():
            
        fine_scan_range = ( 
            res["center"].value - 6 * res["sigma"].value ,
            res["center"].value + 6 * res["sigma"].value
                        ) 
        
        pa.one_pulse_repump(color)
        res = pa.do_ple_scan(lines = 1, in_range=fine_scan_range)
        time.sleep(0.5)
        pa.save_ple(tag = f"{power}_inited_{color}",
        poi_name=poi_name, folder_name=photostability_folder)
        results_poi.update(
            {"photostability":
                    { int(power_steps[idx_no_ple_checked]) :{
                                "charge_initializer": color,
                                "scan_data": ple_data_logic.last_saved_files_paths,
                                "sigma": res["sigma"].value,
                                "sigma_stderr": res["sigma"].stderr
                    }}}
                    )
        if not ple_is_here(res):
            skiped_counter += 1
            continue
        fine_scan_range = ( 
            res["center"].value - 8 * res["sigma"].value ,
            res["center"].value + 8 * res["sigma"].value
                        )

        pa.one_pulse_repump(color)
        res_ = pa.do_ple_scan(lines = 10, in_range=fine_scan_range)
        #fine_scan_range = ( 
        #res["center"].value - scan_range_sat,
        #res["center"].value + scan_range_sat
        #        )
        pa.save_ple(tag = f"{power}_inited_{color}",
        poi_name=poi_name, folder_name=photostability_folder)
        time.sleep(1)
        results_poi.update({"photostability": {
                        "charge_initializer": color,
                        "power": power,
                        "scan_data": ple_data_logic.last_saved_files_paths,
                        "sigma": res["sigma"].value,
                        "sigma_stderr": res["sigma"].stderr
                        }}
                        )
        if idx_no_ple_checked  < idx_no_ple - 2 or skiped_counter == 3:
            break
        #if not ple_is_here(res_):
        #    break

    # take spectrum to estimate SOC
    pa.set_resonant_power(power = 0)
    time.sleep(2)
    ibeam_smart.enable()
    cobolt.enable_modulated()
    cobolt.set_laser_modulated_power(power = 100)
    pa.set_resonant_power(power = 0)
    time.sleep(1)
    pa.take_spectrum()

    pa.save_spectrum(name_tag=f"{poi_name}_blueNgreen", folder_path=folder_defect)
    results_poi["spectrum_data"] = spectrometerlogic.last_saved_path

    spectrometer._mw.data_widget.fit_widget.sigDoFit.emit("DoubleLorentzian")
    time.sleep(0.2)
    # spectrometer.fit_results.params["center_1"].value
    if spectrometer.fit_results is not None:
        params = spectrometer.fit_results.params
        results_poi["SOC, GHz"] = float(speed_of_light / params["center_2"].value - speed_of_light / params["center_1"].value)

    ibeam_smart.disable()
    #cobolt.disable_modulated()
    cobolt.enable_modulated()
    cobolt.set_laser_modulated_power(power = 5)

    with open(os.path.join(folder_defect, f'results_{poi_name}'), 'wb') as handle:
        pickle.dump(results_poi, handle, protocol=pickle.HIGHEST_PROTOCOL)
        

Prepare the modules and functions necessary of the measurements 

In [2]:

def ple_is_here(res, center_sigma = 3e3, amplitude = 1000, sigma_stderr_ratio = 4, amplitude_stderr_ratio=3):
    it_is = True
    if res["center"].stderr is None or res["sigma"].stderr is None or res["sigma"].value is None:
        return False
    if ( res["center"].stderr > center_sigma or 
    res["sigma"].stderr * sigma_stderr_ratio > res["sigma"].value or 
    res["amplitude"].stderr * amplitude_stderr_ratio > res["amplitude"].value or
    res["amplitude"].value < amplitude): 
        #ple is gone.
        return False

    return it_is

# constraints
scan_range_constr = laser_scanner_logic.scanner_constraints.axes["a"].value_range
scanning_optimize_logic._backwards_line_resolution = 20

# high_finesse_wavemeter_remote.start_acquisition()
# switchlogic.set_state("ScanningMode", 'NI')
# time.sleep(1)
# # factor_sigma = 0.509
# pa.go_to_ple_target(scan_range_constr[0])
# time.sleep(5)
# w1 = high_finesse_wavemeter_remote.get_current_wavelength()
# pa.go_to_ple_target(scan_range_constr[1])
# time.sleep(5)
# w2 = high_finesse_wavemeter_remote.get_current_wavelength()
# factor_sigma = (w2 - w1) * 1000 / 26 #GHz 
factor_sigma = 1.05

In [3]:
import sys

sys.setrecursionlimit(10000)

In [4]:
import sys

sys.setrecursionlimit(10000)

In [7]:
laser_controller_remote.etalon_voltage = -7.5

Processing

In [2]:
import numpy as np

In [23]:
ibeam_smart.enable()

In [24]:
ibeam_smart.disable()

In [28]:
cobolt.enable_modulated()

In [27]:
cobolt.set_laser_modulated_power(5)

In [8]:
power_steps

array([300, 261, 231, 204, 177, 156, 138, 120, 105,  93,  85,  78,  70,
        65,  60])